# ResNet

Paper: https://arxiv.org/pdf/1512.03385.pdf

<img src=https://www.researchgate.net/publication/342828449/figure/fig2/AS:911604103278592@1594354763036/ResNet-18-model-architecture-10.png width=600>   
<img src=https://pytorch.org/assets/images/resnet.png width=600>

# Implementation

In [2]:
import os
import torch,torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self,block,in_channels,channels,downsample=False):
        super(ResidualBlock,self).__init__()
        if block=='basic':
            self.conv=nn.Sequential(
                nn.Conv2d(in_channels,channels,3,1+downsample,1),nn.BatchNorm2d(channels),nn.ReLU(),
                nn.Conv2d(channels,channels,3,1,1),nn.BatchNorm2d(channels),
            )
            self.shortcut=nn.Conv2d(in_channels,channels,1,1+downsample) #Option B: projection shortcut
        else: #bottleneck
            self.conv=nn.Sequential(
                nn.Conv2d(in_channels,channels,1,1+downsample),nn.BatchNorm2d(channels),nn.ReLU(),
                nn.Conv2d(channels,channels,3,1,1),nn.BatchNorm2d(channels),nn.ReLU(),
                nn.Conv2d(channels,channels*4,1,1),nn.BatchNorm2d(channels*4)
            )
            self.shortcut=nn.Conv2d(in_channels,channels*4,1,1+downsample)
        self.relu=nn.ReLU()
        
    def forward(self,x):
        r=self.shortcut(x)
        x=self.conv(x)
        x+=r
        x=self.relu(x)
        return x

In [4]:
class ResNet(nn.Module):
    def __init__(self,block,cnt,num_classes=1000):
        super(ResNet,self).__init__()
        self.conv=nn.ModuleDict({'conv1':nn.Sequential(nn.Conv2d(3,64,7,2,3),nn.BatchNorm2d(64),nn.ReLU())})
        self.maxpool=nn.MaxPool2d(3,2,1)
        
        expansion=4-(block=='basic')*3
        for i in range(2,6):
            channels=16<<i
            layer=[ResidualBlock(block,channels//2*expansion if i>2 else 64,channels,i>2)]
            for j in range(1,cnt[i-2]):
                layer.append(ResidualBlock(block,channels*expansion,channels))
            self.conv['conv'+str(i)]=nn.Sequential(*layer)
        
        self.avgpool=nn.AvgPool2d(7)
        self.fc=nn.Sequential(nn.Linear(512*expansion,num_classes))
        
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')
            elif isinstance(m,nn.BatchNorm2d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
        
    def forward(self,x):
        x=self.conv['conv1'](x)
        x=self.maxpool(x)
        for i in range(2,6):
            x=self.conv['conv'+str(i)](x)
        x=self.avgpool(x)
        x=torch.flatten(x,1)
        x=self.fc(x)
        return x

In [5]:
def resnet(num_layers,my=True,pretrained=False,progress=True,**kwargs):
    if my:
        model_dict={18:['basic',[2,2,2,2]],34:['basic',[3,4,6,3]],
                    50:['bottleneck',[3,4,6,3]],101:['bottleneck',[3,4,23,3]],152:['bottleneck',[3,8,36,3]]}
        model=ResNet(*model_dict[num_layers],**kwargs)
    else:
        model_dict={18:models.resnet18,34:models.resnet34,
                    50:models.resnet50,101:models.resnet101,152:models.resnet152}
        model=model_dict[num_layers](pretrained=pretrained,progress=progress,**kwargs)
    return model

# Run

In [6]:
import import_ipynb
from ImageLoader import SimpleLoader
import tnt

importing Jupyter notebook from ImageLoader.ipynb
importing Jupyter notebook from tnt.ipynb


In [7]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE=64
VALID_SIZE=0.1
NUM_EPOCHS=20
PATH='checkpoint/ResNet'

## Train

In [8]:
sl=SimpleLoader(dataset='CIFAR100',batch_size=BATCH_SIZE,crop_size=224,split=True)
num_classes=sl.GetNumClasses()
train_loader=sl.GetTrainLoader()
valid_loader=sl.GetValidLoader()

Files already downloaded and verified
Files already downloaded and verified


In [9]:
criterion=nn.CrossEntropyLoss().to(device)

In [17]:
model1=resnet(num_layers=18,num_classes=num_classes).to(device)
#optimizer=optim.SGD(params=model1.parameters(),lr=0.01,momentum=0.9,weight_decay=0.0005)
optimizer=optim.Adam(params=model1.parameters(),lr=0.0001)
scheduler=optim.lr_scheduler.StepLR(optimizer,step_size=30,gamma=0.1)

In [19]:
tnt.train(model1,device,NUM_EPOCHS,train_loader,valid_loader,criterion,optimizer,scheduler,save=1,path=PATH)

Epoch  1 	 train_loss: 3.69605 	 top1_acc: 21.53% 	 top5_acc: 50.22%


Epoch  2 	 train_loss: 3.01217 	 top1_acc: 29.55% 	 top5_acc: 61.74%


Epoch  3 	 train_loss: 2.56496 	 top1_acc: 37.90% 	 top5_acc: 70.41%


Epoch  4 	 train_loss: 2.23444 	 top1_acc: 43.43% 	 top5_acc: 74.90%


Epoch  5 	 train_loss: 1.97345 	 top1_acc: 48.22% 	 top5_acc: 79.09%


...

Epoch 16 	 train_loss: 0.62498 	 top1_acc: 60.70% 	 top5_acc: 88.00%


Epoch 17 	 train_loss: 0.55128 	 top1_acc: 61.48% 	 top5_acc: 87.36%


Epoch 18 	 train_loss: 0.47993 	 top1_acc: 61.20% 	 top5_acc: 87.06%


Epoch 19 	 train_loss: 0.42530 	 top1_acc: 61.60% 	 top5_acc: 87.56%


Epoch 20 	 train_loss: 0.36895 	 top1_acc: 61.86% 	 top5_acc: 87.34%


In [10]:
model2=resnet(num_layers=50,num_classes=num_classes).to(device)
#optimizer=optim.SGD(params=model2.parameters(),lr=0.01,momentum=0.9,weight_decay=0.0005)
optimizer=optim.Adam(params=model2.parameters(),lr=0.0001)
scheduler=optim.lr_scheduler.StepLR(optimizer,step_size=30,gamma=0.1)

In [13]:
tnt.train(model2,device,NUM_EPOCHS,train_loader,valid_loader,criterion,optimizer,scheduler,save=1,path=PATH)

KeyboardInterrupt: 

## Test

In [12]:
from ensemble import Ensemble

importing Jupyter notebook from ensemble.ipynb


In [14]:
model1=resnet(num_layers=18,num_classes=num_classes).to(device)
model1.load_state_dict(torch.load(os.path.join(PATH,'CIFAR100_res18_e20_best')))
model2=resnet(num_layers=50,num_classes=num_classes).to(device)
model2.load_state_dict(torch.load(os.path.join(PATH,'CIFAR100_res50_e20_best')))

<All keys matched successfully>

In [15]:
model=Ensemble(model1,model2)

In [16]:
test_loader=sl.GetTestLoader()
tnt.test(model,device,test_loader,criterion)

loss: 3.99340 	 top1_acc: 64.30% 	 top5_acc: 88.55%
